# Security Manual Fine Tuning

## 1. 필요한 라이브러리 설치

In [ ]:
!pip install -q transformers datasets accelerate huggingface_hub

## 2. 모델 및 토크나이저 불러오기

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# 1. 모델 불러오기
model_name = "kingkim/kodialogpt_v1.1_SecurityManual"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/64.6k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/891k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/496k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## 2. 데이터셋 불러오기

In [ ]:
dataset = load_dataset("kingkim/DS_Building_SecurityManual_V5")  # 사용자 정의 데이터셋 사용

README.md:   0%|          | 0.00/266 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.36k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/116 [00:00<?, ? examples/s]

## 3. 프롬프트 생성 함수 정의와 데이터 셋 전처리


In [ ]:
# 3. 프롬프트 생성 함수 정의
def generate_prompt(data_point):
    # 데이터셋에서 질문과 답변을 분리하여 프롬프트 생성
    if isinstance(data_point, str):  # data_point가 문자열인지 확인
        if '\n' in data_point:
            input_text, output_text = data_point.split('\n', 1)
        else:
            input_text = data_point
            output_text = ""  # 답변이 없을 경우 기본값
        return f"Q: {input_text} \nA: {output_text}"
    else:
        raise TypeError("data_point가 문자열이어야 합니다.")

# 4. 데이터셋 전처리 함수 수정
def preprocess_function(examples):
    # 배치 단위로 examples가 리스트 형태로 넘어옵니다.
    # 따라서 각 예제를 개별적으로 처리한 후 리스트 형태로 반환해야 합니다.
    prompts = [generate_prompt(example) for example in examples['text']]  # 각 예제를 개별적으로 처리
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(prompts, truncation=True, padding="max_length", max_length=512)

# map 함수로 전처리 적용
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.shuffle(seed=42)

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

In [ ]:
# 3. 프롬프트 생성 함수 정의
def generate_prompt(data_point):
    # 데이터셋에서 질문과 답변을 분리하여 프롬프트 생성
    # data_point가 dictionary인지 확인합니다.
    if isinstance(data_point, dict):
        # 'input' 또는 'content' 와 같은 실제 column 이름으로 변경합니다.
        input_text = data_point.get("input", "")  # input column에서 가져오기, 또는 "content" 등으로 변경
        output_text = data_point.get("output", "")  # output column에서 가져오기, 또는 "summary" 등으로 변경
        return f"Q: {input_text} \nA: {output_text}"
    # data_point가 문자열인지 확인합니다.
    elif isinstance(data_point, str):
        if '\n' in data_point:
            input_text, output_text = data_point.split('\n', 1)
        else:
            input_text = data_point
            output_text = ""  # 답변이 없을 경우 기본값
        return f"Q: {input_text} \nA: {output_text}"
    else:
        raise TypeError("data_point가 문자열 또는 딕셔너리여야 합니다.")

# 4. 데이터셋 전처리 함수 수정
def preprocess_function(examples):
    # 배치 단위로 examples가 리스트 형태로 넘어옵니다.
    # 따라서 각 예제를 개별적으로 처리한 후 리스트 형태로 반환해야 합니다.
    # examples['input'] 또는 examples['content'] 와 같은 실제 column 이름으로 변경합니다.
    prompts = [generate_prompt(example) for example in examples['input']]  # 각 예제를 개별적으로 처리, 또는 "content" 등으로 변경
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(prompts, truncation=True, padding="max_length", max_length=512)

# map 함수로 전처리 적용
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.shuffle(seed=42)

## 4. 학습 설정

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # Colab 환경에 맞춰 배치 사이즈 조정
    gradient_accumulation_steps=4,
    num_train_epochs=5,  # 학습 에폭 수
    learning_rate=2e-4,
    logging_dir='./logs',
    save_strategy="epoch",  # 매 에폭마다 모델 저장
    evaluation_strategy="epoch",  # 매 에폭마다 평가
    logging_steps=10,
    load_best_model_at_end=True
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Masked Language Modeling이 아님
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 5. Trainer 설정

In [ ]:
# 데이터셋을 학습/평가로 분리
train_test_split = tokenized_dataset['train'].train_test_split(test_size=0.2)  # 80% 학습, 20% 평가
train_data = train_test_split['train']
eval_data = train_test_split['test']

# Trainer 설정에서 eval_dataset 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,  # 학습 데이터
    eval_dataset=eval_data,    # 평가 데이터
    data_collator=data_collator,
    tokenizer=tokenizer,
)

## 6. 모델 파인튜닝

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,2.211400,1.583810
2,0.603300,0.982826
4,0.242300,0.908312


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=55, training_loss=0.7861400214108554, metrics={'train_runtime': 139.8192, 'train_samples_per_second': 3.29, 'train_steps_per_second': 0.393, 'total_flos': 114968494080000.0, 'train_loss': 0.7861400214108554, 'epoch': 4.782608695652174})

## 7. 모델 저장 및 Hugging Face에 업로드

In [ ]:
# 8. 모델 저장 및 Hugging Face에 업로드
model_directory = "./finetuned_kodialogpt_v3.2_SecurityManual"

# 모델과 토크나이저를 로컬 디렉토리에 저장
model.save_pretrained(model_directory)
tokenizer.save_pretrained(model_directory)

# Hugging Face에 업로드 (로그인 필요)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("kingkim/kodialogpt_v3.2_SecurityManual", token="YOUR_Hugging Face_TOKEN")
tokenizer.push_to_hub("kingkim/kodialogpt_v3.2_SecurityManual", token="YOUR_Hugging Face_TOKEN")

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kingkim/kodialogpt_v3.2_SecurityManual/commit/a6714e0a7df26540ed44808ac7ce2126c486775a', commit_message='Upload tokenizer', commit_description='', oid='a6714e0a7df26540ed44808ac7ce2126c486775a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# trainer도 Hugging Face에 업로드 가능
trainer.push_to_hub("kingkim/kodialogpt_v3.2_SecurityManual", token="YOUR_Hugging Face_TOKEN")

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kingkim/results/commit/39c7907776a9b9f16c5db6ffe6497c7ad20efca5', commit_message='kingkim/kodialogpt_v3.2_SecurityManual', commit_description='', oid='39c7907776a9b9f16c5db6ffe6497c7ad20efca5', pr_url=None, pr_revision=None, pr_num=None)